In [1]:
from typing import List, Dict, Tuple, Type, Any

In [2]:
class MisMatchedBrackets(Exception):
    pass
class InvalidNode(Exception):
    pass

In [4]:
class Node:
    def __init__(self, val:str, left = None, right = None, typ:str = 'V'):
        self.val = val
        self.typ = typ
        if typ != 'V' and typ != 'INVALID' and typ != 'EMPTY' and left == None and right == None:
            raise InvalidNode(f'Non-Variable nodes {val} should have atleast one child')
        self.left = left
        self.right = right
    
    @property 
    def is_var(self):
        return self.typ == 'V'
    
    def __str__(self):
        st = []
        st += str(self.val)
        if self.typ != 'V':
            st += '['
            for node in [self.left, self.right]:
                st += node.__str__()
                st += ','
            st[-1] = ']'
        return ''.join(st)

In [5]:
def detect_mismatched_brackets(NNF: str):
    st = []
    le = len(NNF)
    for i in range(le):
        char = NNF[i]
        if char == '(':
            st.append(i)
        elif char == ')':
            if st == []:
                raise MisMatchedBrackets(f'Extra closing bracket at index {i+1} in file')
            st.pop()
        
    if st != []:
        raise MisMatchedBrackets(f'Unpaired opening bracket at index {st[0]+1} in file')

In [6]:
def remove_brackets(NNF: str):
    li = ['']
    le = len(NNF)
    i = 0
    while i < le:
        if NNF[i] == '(':
            counter = 0
            flag = True
            for j in range(i+1, le):
                flag = False
                if NNF[j] == '(':
                    counter += 1
                if NNF[j] == ')':
                    if counter == 0:
                        sub_list = remove_brackets(NNF[i+1:j])
                        if li[-1] == '':
                            li.pop()
                        if li == []:
                            li.extend(sub_list)
                        else:
                            li.append(sub_list)
                        li.append('')
                        i = j
                        break
                    counter -= 1
        else:
            li[-1] += NNF[i]
        i += 1
    if li[-1] == '':
        li.pop()
    return li

In [7]:
def already_bracketed(NNF: str):
    print(NNF)
    st = []
    le = len(NNF)
    for i in range(le):
        char = NNF[i]
        if char == '(':
            st.append(char)
            flag = True
        if char == ')':
            st.pop()
            if i != le-1 and st == []:
                return False
    return True
    

In [17]:
operators = {'+', '.'}
def add_brackets(NNF):
    print(NNF)
    if NNF[0] == '(' and NNF[-1] == ')':
        try:
            detect_mismatched_brackets(NNF[1:-1])
            print('here')
            return add_brackets(NNF[1:-1])
        except:
            pass
    if len(NNF) == 1:
        return NNF
    for i in range(len(NNF)):
        if NNF[i] in operators:
            return '(' + add_brackets(NNF[:i]) +  ')' + NNF[i] + '(' + add_brackets(NNF[i+1:]) +  ')'
    return NNF

In [29]:
def printDAG(filename):
    f = open(filename)
    NNF = f.readline()
    detect_mismatched_brackets(NNF)
    NNF = NNF.replace(' ', '').replace('\n', '')
    # .replace('.', ').(').replace('+', ')+(') + ')'
    new_NNF = add_brackets(NNF)
    print(new_NNF)

In [30]:
printDAG('./nnf/7.nnf')

1+2+3+4+-5+-1+(-2+(-1))
1
2+3+4+-5+-1+(-2+(-1))
2
3+4+-5+-1+(-2+(-1))
3
4+-5+-1+(-2+(-1))
4
-5+-1+(-2+(-1))
-5
-1+(-2+(-1))
-1
(-2+(-1))
here
-2+(-1)
-2
(-1)
here
-1
(1)+((2)+((3)+((4)+((-5)+((-1)+((-2)+(-1)))))))


In [31]:
printDAG('./nnf/8.nnf')

1.2.3.4.-5.-1.(-2.(-1))
1
2.3.4.-5.-1.(-2.(-1))
2
3.4.-5.-1.(-2.(-1))
3
4.-5.-1.(-2.(-1))
4
-5.-1.(-2.(-1))
-5
-1.(-2.(-1))
-1
(-2.(-1))
here
-2.(-1)
-2
(-1)
here
-1
(1).((2).((3).((4).((-5).((-1).((-2).(-1)))))))


In [32]:
printDAG('./nnf/2.nnf')

1.(2+3.5+-1.3)+4.6+12+15.16+(17.18)+((-1))
1
(2+3.5+-1.3)+4.6+12+15.16+(17.18)+((-1))
(2
3.5+-1.3)+4.6+12+15.16+(17.18)+((-1))
3
5+-1.3)+4.6+12+15.16+(17.18)+((-1))
5
-1.3)+4.6+12+15.16+(17.18)+((-1))
-1
3)+4.6+12+15.16+(17.18)+((-1))
3)
4.6+12+15.16+(17.18)+((-1))
4
6+12+15.16+(17.18)+((-1))
6
12+15.16+(17.18)+((-1))
12
15.16+(17.18)+((-1))
15
16+(17.18)+((-1))
16
(17.18)+((-1))
(17
18)+((-1))
18)
((-1))
here
(-1)
here
-1
(1).(((2)+((3).((5)+((-1).((3))+((4).((6)+((12)+((15).((16)+(((17).((18))+(-1)))))))))))))


In [21]:
# st = []
# for i in range(len(NNF)):
#     char = NNF[i]
#     if char == '(':
#         st.append(char)
#     elif char == ')':
#         st.pop()
#         if st == []:

In [22]:
def convert_to_DAG(NNF: str) -> Node:
    # print(NNF)
    if len(NNF) == 0:
        return Node(val='EMPTY', typ='EMPTY', left=None, right=None)
    if len(NNF) <= 2:
        # print('2: ', NNF)
        return Node(val=NNF, typ='V')
    if len(NNF) <= 4:
        # print('4: ', NNF)
        return Node(val=NNF[1:-1], typ='V')
    for i in range(len(NNF)):
        char = NNF[i]
        if char in operators:
            # print(f'left: {NNF[:i]}, right: {NNF[i+2:-1]}')
            return Node(val=char, typ=char, left=convert_to_DAG(NNF[:i]), right=convert_to_DAG(NNF[i+2:-1]))
    print('E', NNF)
    # return Node(val='INVALID', typ='INVALID', left=None, right=None)
        

            
    

In [23]:
# remove_brackets(NNF)

In [24]:
root = convert_to_DAG(new_NNF)

In [25]:
print(root)

+[1,+[2,+[3,+[4,+[-5,+[-1,+[-2,-1]]]]]]]
